# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233451815690                   -0.50    7.0
  2   -7.250039144151       -1.78       -1.40    1.0
  3   -7.251272968470       -2.91       -2.02    4.0
  4   -7.251195533226   +   -4.11       -2.07    4.0
  5   -7.251319866886       -3.91       -2.53    3.0
  6   -7.251336766768       -4.77       -2.96    3.0
  7   -7.251338777086       -5.70       -3.98    2.0
  8   -7.251338791719       -7.83       -4.13    4.0
  9   -7.251338798570       -8.16       -4.86    3.0
 10   -7.251338798685       -9.94       -5.37    3.0
 11   -7.251338798702      -10.78       -5.86    3.0
 12   -7.251338798704      -11.62       -6.40    2.0
 13   -7.251338798705      -12.90       -7.07    3.0
 14   -7.251338798705      -14.35       -7.40    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05761833890173986
[ Info: Arnoldi iteration step 2: normres = 0.4263062033783244
[ Info: Arnoldi iteration step 3: normres = 0.9607306557542182
[ Info: Arnoldi iteration step 4: normres = 0.2987324446826641
[ Info: Arnoldi iteration step 5: normres = 0.15391002859448014
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.44e-02, 1.45e-02, 9.07e-02, 1.07e-01, 6.00e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5626718718964424
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.47e-03, 1.07e-02, 5.25e-01, 1.83e-01, 3.66e-02)
[ Info: Arnoldi iteration step 7: normres = 0.18511465949893796
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (7.45e-04, 1.55e-01, 1.79e-02, 4.84e-02, 8.17e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09363385866780242
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.10e-